In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/Edinburg/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/1508-e-optimum-dr-n-edinburg-tx-78542-2056938500',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Sophora-344817293',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Althea-344817294',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Rosemary-344817295',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Cassia-344817296',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Jasmine-344817297',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Poinsettia-344817300',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Duranta-344817301',
 'https://www.trulia.com/builder-community-plan/Alberta-Heights-Lantana-344817302',
 'https://www.trulia.com/home/5506-harwell-ln-edinburg-tx-78542-344797212',
 'https://www.trulia.com/builder-community-plan/Silos-At-La-Sienna-Espada-2053900767',
 'https://www.trulia.com/home/3610-valeria-st-edinburg-tx-78541-303757141',
 'http

### Get data from links

In [7]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [8]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/1508-e-optimum-dr-n-edinburg-tx-78542-2056938500


### Get request and Soup Object

In [9]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [10]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [12]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['1508 E  Optimum Dr   N',
  'Sophora Plan in Alberta Heights by Hakes Brothers',
  'Althea Plan in Alberta Heights by Hakes Brothers',
  'Rosemary Plan in Alberta Heights by Hakes Brothers',
  'Cassia Plan in Alberta Heights by Hakes Brothers',
  'Jasmine Plan in Alberta Heights by Hakes Brothers',
  'Poinsettia Plan in Alberta Heights by Hakes Brothers',
  'Duranta Plan in Alberta Heights by Hakes Brothers',
  'Lantana Plan in Alberta Heights by Hakes Brothers',
  '5506 Harwell Ln',
  'Espada Plan in Silos at La Sienna by Esperanza Homes',
  '3610 Valeria St',
  '4105 Burr Dr',
  '1309 Prosperity Dr',
  '2080 Eagle Landing Dr',
  '3911 N  Salinas St',
  '3624 Lasso Trl',
  'San Lorenzo II Plan in Silos at La Sienna by Esperanza Homes',
  '35206 Longhorn Dr',
  '2903 Allen Dr',
  'Allamanda Plan in Alberta Heights by Hakes Brothers',
  '821 N  Buxton Ave',
  '1733 Pinnacle Dr',
  '4306 Harmony Ln',
  '7006 Tex Mex Rd',
  '1911 Woods Dr W',
  '7501 Stenhouse St',
  '1411 E 

In [13]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,1508 E Optimum Dr N,2 Baths,"1,594 sqft",2022,2 Car Garage,"$312,000"
1,Sophora Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,526 sqft",2024,Garage,"$285,990+"
2,Althea Plan in Alberta Heights by Hakes Brothers,3 Baths,"2,118 sqft",2024,Garage,"$338,990+"
3,Rosemary Plan in Alberta Heights by Hakes Brot...,3 Baths,"2,055 sqft",2024,Garage,"$329,990+"
4,Cassia Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,939 sqft",2024,Garage,"$315,990+"
5,Jasmine Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,781 sqft",2024,Garage,"$309,990+"
6,Poinsettia Plan in Alberta Heights by Hakes Br...,3 Baths,"2,675 sqft",2024,Garage,"$375,990+"
7,Duranta Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,675 sqft",2024,Garage,"$299,990+"
8,Lantana Plan in Alberta Heights by Hakes Brothers,3 Baths,"2,045 sqft",2024,Garage,"$325,990+"
9,5506 Harwell Ln,1 Bath,768 sqft,2022,No Info,"$109,500"


In [14]:
# Add location to dataframe
df['Location'] = 'Edinburg'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,1508 E Optimum Dr N,2 Baths,"1,594 sqft",2022,2 Car Garage,"$312,000",Edinburg
1,Sophora Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,526 sqft",2024,Garage,"$285,990+",Edinburg
2,Althea Plan in Alberta Heights by Hakes Brothers,3 Baths,"2,118 sqft",2024,Garage,"$338,990+",Edinburg
3,Rosemary Plan in Alberta Heights by Hakes Brot...,3 Baths,"2,055 sqft",2024,Garage,"$329,990+",Edinburg
4,Cassia Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,939 sqft",2024,Garage,"$315,990+",Edinburg
5,Jasmine Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,781 sqft",2024,Garage,"$309,990+",Edinburg
6,Poinsettia Plan in Alberta Heights by Hakes Br...,3 Baths,"2,675 sqft",2024,Garage,"$375,990+",Edinburg
7,Duranta Plan in Alberta Heights by Hakes Brothers,2 Baths,"1,675 sqft",2024,Garage,"$299,990+",Edinburg
8,Lantana Plan in Alberta Heights by Hakes Brothers,3 Baths,"2,045 sqft",2024,Garage,"$325,990+",Edinburg
9,5506 Harwell Ln,1 Bath,768 sqft,2022,No Info,"$109,500",Edinburg


In [15]:
df.to_excel('dataframe_edinburg.xlsx', index=False)